# 2. Feature Analysis, Model Training, and Evaluation

This notebook takes the events data (with heuristic labels from Notebook 1) and the full time-series CPAP data to:
1.  Extract detailed features for each event using `src.feature_engineering`.
2.  Analyze the engineered features.
3.  Train a classification model using `src.classification_model`.
4.  Evaluate the model's performance on our heuristically labeled data.
5.  Discuss potential iterations and improvements.

## 2.1 Setup and Data Loading

Load preprocessed time-series data and the event data with heuristic labels.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add src directory to Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data_loader import load_cpap_data, resample_data # For loading full signal
from src.preprocessing import butterworth_filter, calculate_rolling_baseline # If re-running parts
from src.feature_engineering import extract_features_for_event
from src.classification_model import train_classification_model

plt.rcParams['figure.figsize'] = (12, 4)
sns.set_theme(style="whitegrid")

In [ ]:
# --- Load the full time-series data (preprocessed if possible, or re-preprocess) ---
# This assumes you have saved the resampled/preprocessed df from Notebook 1 or can reload raw and process.
# For simplicity, we'll reload the dummy data and do minimal preprocessing again.

DUMMY_DATA_FILENAME = "dummy_cpap_data.csv" # From Notebook 1
cpap_data_filepath = f'../data/{DUMMY_DATA_FILENAME}'
SAMPLING_FREQ_HZ = 25

raw_cpap_df = load_cpap_data(cpap_data_filepath,
                               timestamp_col='Timestamp',
                               flow_rate_col='FlowRate',
                               pressure_col='Pressure',
                               leak_rate_col='LeakRate',
                               # Optional cols if used by feature engineering directly
                               minute_vent_col='MinuteVent',
                               resp_rate_col='RespRate',
                               tidal_vol_col='TidalVol'
                              )

if raw_cpap_df is None:
    raise FileNotFoundError(f"Ensure '{DUMMY_DATA_FILENAME}' exists in '../data/' from Notebook 1.")

full_signal_df = resample_data(raw_cpap_df, target_freq_hz=SAMPLING_FREQ_HZ)
if full_signal_df is None:
    raise ValueError("Failed to resample data.")

# Minimal preprocessing needed for feature engineering context
full_signal_df['flow_rate_filtered'] = butterworth_filter(
    full_signal_df['flow_rate'], 'lowpass', 2.0, SAMPLING_FREQ_HZ
)
full_signal_df['pressure_filtered'] = butterworth_filter(
    full_signal_df['pressure'], 'lowpass', 1.0, SAMPLING_FREQ_HZ # Smooth pressure a bit
)

print("Full time-series signal data loaded and minimally processed.")
full_signal_df[['flow_rate_filtered', 'pressure_filtered']].info()

# --- Load the events DataFrame with heuristic labels ---
events_labeled_filepath = '../data/events_with_heuristic_labels.csv'
try:
    events_df = pd.read_csv(events_labeled_filepath,
                            parse_dates=['event_start_time', 'event_end_time'])
except FileNotFoundError:
    print(f"Error: '{events_labeled_filepath}' not found. Please run Notebook 1 first.")
    events_df = pd.DataFrame() # Assign empty to prevent further errors

if not events_df.empty:
    print(f"\nLoaded {len(events_df)} events with heuristic labels.")
    print(events_df[['event_start_time', 'event_type', 'heuristic_label']].head())
    print("\nLabel distribution:")
    print(events_df['heuristic_label'].value_counts())
else:
    print("Event data is empty. Cannot proceed with feature engineering and model training.")

## 2.2 Feature Engineering for Labeled Events

Iterate through each labeled event and extract features using `extract_features_for_event`.

In [ ]:
all_event_features_list = []
if not events_df.empty and full_signal_df is not None:
    for idx, event_row in events_df.iterrows():
        # Ensure event_row.name is set for feature engineering function if it relies on it
        event_row.name = idx 
        
        # Extract features for this event
        # Pass relevant signals from full_signal_df
        current_event_features = extract_features_for_event(
            event_row=event_row,
            flow_series=full_signal_df['flow_rate_filtered'], 
            pressure_series=full_signal_df['pressure_filtered'],
            sampling_freq_hz=SAMPLING_FREQ_HZ,
            pre_event_window_s=30,
            post_event_window_s=30
        )
        all_event_features_list.append(current_event_features)
    
    features_engineered_df = pd.DataFrame(all_event_features_list)
    
    # Merge heuristic labels back into the features dataframe
    # `event_id` in features_engineered_df should correspond to the index of events_df
    features_engineered_df = features_engineered_df.set_index('event_id')
    features_engineered_df = features_engineered_df.join(events_df[['heuristic_label']])
    
    print(f"\nEngineered features for {len(features_engineered_df)} events.")
    print(features_engineered_df.head().T)
    
    # Save the engineered features
    engineered_features_path = '../data/engineered_features_with_labels.csv'
    features_engineered_df.to_csv(engineered_features_path)
    print(f"\nSaved engineered features to: {engineered_features_path}")
else:
    print("Skipping feature engineering as event data or signal data is missing.")
    features_engineered_df = pd.DataFrame()

## 2.3 Feature Analysis (Basic)

Let's look at some feature distributions across our heuristic labels. This can give clues about which features might be discriminative.

In [ ]:
if not features_engineered_df.empty and 'heuristic_label' in features_engineered_df.columns:
    # Select a few example features for visualization
    # These names come from `feature_engineering.py` output
    example_features_to_plot = [
        'flow_during_abs_mean', 
        'pressure_rise_from_pre_to_during',
        'hypopnea_flow_shape_flattening_idx',
        'recovery_peak_flow_post5s',
        'flow_pre_periodicity_lf_hf_ratio',
        'flow_pre_coeff_var'
    ]
    
    # Drop rows where heuristic_label is NaN if any (should be handled by model training too)
    plot_df = features_engineered_df.dropna(subset=['heuristic_label'])
    # Also, for some plots, drop rows where the feature itself is NaN to avoid errors

    num_features_to_plot = len(example_features_to_plot)
    plt.figure(figsize=(15, num_features_to_plot * 4))
    for i, feature_name in enumerate(example_features_to_plot):
        if feature_name not in plot_df.columns:
            print(f"Warning: Feature '{feature_name}' not found in DataFrame for plotting.")
            continue
        
        plt.subplot(num_features_to_plot, 1, i + 1)
        # For violinplot, it's better to have non-NaN feature values
        feature_plot_df = plot_df[[feature_name, 'heuristic_label']].dropna(subset=[feature_name])
        if not feature_plot_df.empty:
            sns.violinplot(x='heuristic_label', y=feature_name, data=feature_plot_df, 
                           order=['likely_central', 'ambiguous', 'likely_obstructive'])
            plt.title(f'Distribution of {feature_name} by Heuristic Label')
        else:
            plt.text(0.5, 0.5, 'Not enough data for this feature/label combination', ha='center')
            plt.title(f'{feature_name} (No Data)')
            
    plt.tight_layout()
    plt.show()
    
    # Correlation heatmap (on a subset of features to keep it readable)
    numeric_cols = plot_df.select_dtypes(include=np.number).columns
    # Select a subset of numeric_cols for heatmap if too many, e.g., first 15-20, or those plotted above
    heatmap_features = [f for f in example_features_to_plot if f in numeric_cols]
    if len(heatmap_features) > 1:
        plt.figure(figsize=(10, 8))
        correlation_matrix = plot_df[heatmap_features].corr()
        sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
        plt.title('Correlation Matrix of Selected Features')
        plt.show()
    else:
        print("Not enough numeric features selected for correlation heatmap.")
else:
    print("Skipping feature analysis as engineered features or labels are missing.")

## 2.4 Model Training and Evaluation

Train a classifier using the engineered features and the heuristic labels.

In [ ]:
if not features_engineered_df.empty and 'heuristic_label' in features_engineered_df.columns:
    # Drop events labeled as 'ambiguous' if we want a 2-class problem for initial model
    # Or, treat 'ambiguous' as a third class if model supports it well / if desired.
    # For this example, let's try to classify obstructive vs central, and exclude ambiguous for training.
    trainable_df = features_engineered_df[features_engineered_df['heuristic_label'].isin(['likely_obstructive', 'likely_central'])]
    
    if len(trainable_df) > 10: # Need enough samples to train
        print(f"Training model on {len(trainable_df)} 'likely_obstructive' or 'likely_central' events.")
        
        # Select model type
        model_type_to_train = 'random_forest' # 'random_forest', 'logistic_regression', 'svm'
        
        trained_model_pipeline, test_predictions_df, metrics = train_classification_model(
            features_df=trainable_df.copy(), # Pass a copy
            target_column='heuristic_label',
            model_type=model_type_to_train,
            test_size=0.25, # Use 25% for test set
            random_state=42,
            n_folds=5 # 5-fold cross-validation on training part
        )
        
        print(f"\n--- {model_type_to_train} Model Evaluation Summary ---")
        print(f"Test Set Accuracy: {metrics['accuracy']:.4f}")
        print(f"Mean CV Accuracy on Training Data: {metrics['cv_mean_accuracy']:.4f} (std: {metrics['cv_std_accuracy']:.4f})")
        print("\nTest Set Classification Report:\n", metrics['classification_report_str'])
        print("\nTest Set Confusion Matrix:")
        print(metrics['confusion_matrix_df'])
        
        # Display feature importances if it's a RandomForest model
        if model_type_to_train == 'random_forest' and 'classifier' in trained_model_pipeline.named_steps:
            rf_model = trained_model_pipeline.named_steps['classifier']
            # Get feature names after imputation/scaling (if possible, depends on pipeline structure)
            # For simplicity, assume column order is preserved or use original feature names from X_train
            # Need to get feature names from the step before classifier
            # This requires knowing the columns used for training (X used in train_classification_model)
            
            # Re-create the X that was used for training, to get column names
            temp_X = trainable_df.drop(columns=['heuristic_label', 'event_id', 
                                                 'event_type_original', 'excluded_due_to_leak_original'], 
                                       errors='ignore')
            numeric_features_used = temp_X.select_dtypes(include=np.number).columns.tolist()
            
            if hasattr(rf_model, 'feature_importances_') and numeric_features_used:
                importances = rf_model.feature_importances_
                feature_importance_df = pd.DataFrame({
                    'feature': numeric_features_used,
                    'importance': importances
                }).sort_values(by='importance', ascending=False)
                
                plt.figure(figsize=(10, 8))
                sns.barplot(x='importance', y='feature', data=feature_importance_df.head(20), palette='viridis')
                plt.title('Top 20 Feature Importances (Random Forest)')
                plt.tight_layout()
                plt.show()
            else:
                print("Could not retrieve feature importances for Random Forest.")
    else:
        print("Not enough 'likely_obstructive' or 'likely_central' labeled events to train a model.")
else:
    print("Skipping model training as feature data is missing or labels are not present.")

## 2.5 Discussion and Next Steps

The model's performance here is heavily dependent on the quality and quantity of the **heuristic labels** from Notebook 1. 

**Potential Next Steps for Iteration:**
1.  **Refine Heuristic Labels:** 
    *   Go back to Notebook 1 (`01_Initial_Data_Exploration_and_Labeling.ipynb`).
    *   Inspect more events, especially those misclassified by the current model (if predictions are available on the full dataset) or those where the model is uncertain.
    *   Improve the heuristic rules. This might involve using some of the engineered features (e.g., if `flow_pre_periodicity_lf_hf_ratio` is high, it's more likely central).
    *   Consider an active learning loop: train a model, use it to find the most uncertain samples, label those, and retrain.
2.  **Feature Engineering & Selection:**
    *   Are there other features that could be more discriminative? (e.g., more detailed breath-shape analysis from `breath_detection.py`, more sophisticated pressure change metrics).
    *   Analyze feature importance (shown for Random Forest above). Are some features not contributing much or too correlated?
    *   Consider dimensionality reduction (e.g., PCA) if there are very many features.
3.  **Model Tuning:**
    *   Try different model types (e.g., Gradient Boosting like XGBoost or LightGBM, which often perform well).
    *   Perform hyperparameter tuning for the chosen model (e.g., using `GridSearchCV` or `RandomizedSearchCV` from scikit-learn).
4.  **Address 'Ambiguous' Events:**
    *   Decide on a strategy: 
        *   Keep them excluded from training a binary (obstructive/central) model.
        *   Try to label them if patterns emerge.
        *   Train a 3-class model (obstructive, central, ambiguous) if sufficient 'ambiguous' examples with consistent characteristics can be identified.
        *   Use the binary model's probability scores: if probabilities for obstructive/central are both low (e.g., around 0.5), the model is uncertain, and the event could be flagged as ambiguous.
5.  **Data Augmentation (if applicable and careful):** If one class is very underrepresented, explore techniques, but this is advanced and must be done cautiously to avoid unrealistic samples.
6.  **External Validation:** Ultimately, validation against expert-scored Polysomnography (PSG) data would be needed to assess true clinical performance. The current evaluation is against our own heuristic labels.